In [1]:
using Pkg
Pkg.activate("..")
using Dispersions
using Test
using FFTW
include("../test/helper_functions.jl")

  Activating project at `~/Codes/Dispersions.jl`
[ Info: Precompiling Dispersions [9451dd48-6b2a-4bde-a443-9ec270a37806]


fcc_dispersion (generic function with 1 method)

In [24]:
Ns = 4

v_full = collect(gen_sampling(Dispersions.cP, 2, Ns))
q0 = Tuple(findfirst(x-> all(x .≈ 0), v_full))

LoadError: UndefVarError: `gen_sampling` not defined

In [12]:
Ns = 4


gr = "fcc-1.7"
kG = gen_kGrid(gr, Ns)

v_full = collect(Dispersions.gen_sampling(grid_type(kG), 2, Ns))
q0 = Tuple(findfirst(x-> all(x .≈ 0), v_full))

arr1 = randn(ComplexF64, gridshape(kG))
arr2 = randn(ComplexF64, gridshape(kG))
arr1_sym = expandKArr(kG, reduceKArr(kG, arr1))
arr2_sym = expandKArr(kG, reduceKArr(kG, arr2))
arr1_c = deepcopy(arr1_sym)
arr2_c = deepcopy(arr2_sym);

In [13]:
res_pp, dbg_i_pp, dbg_val_pp = naive_conv(v_full, v_full, pp=true, q0, round_entries=false);
res_ph, dbg_i, dbg_val = naive_conv(v_full, v_full, pp=false, q0, round_entries=false);

In [14]:
q_list = map(el->check_q(el, Ns), dbg_val);
# preprocess rounds all entries in order for unqiue to work
q_list_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list)
q_list_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pre))
# Extract only q vector (since we checked that only one unique exists
q_list_single = map(x->x[1], q_list_pre)
# Test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val) .== Ns*Ns)

Test Failed at In[14]:10
  Expression: all(map((x->begin
                all(x[1] .≈ x[2])
            end), zip(q_list_single, v_full)))



LoadError: [91mThere was an error during testing[39m

In [15]:
q_list_pp = map(el->check_q(el, Ns, pp=true), dbg_val_pp);
# preprocess rounds all entries in order for unqiue to work
q_list_pp_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list_pp)
q_list_pp_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pp_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pp_pre))
# extract only q vector (since we checked that only one unique exists
q_list_pp_single = map(x->x[1], q_list_pp_pre)
# test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_pp_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pp_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val_pp) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val_pp) .== Ns*Ns)

Test Failed at In[15]:10
  Expression: all(map((x->begin
                all(x[1] .≈ x[2])
            end), zip(q_list_pp_single, v_full)))



LoadError: [91mThere was an error during testing[39m

In [16]:
conv_ph_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=false);
conv_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG,arr2_sym), crosscorrelation=true)
conv_ph_fft_test = circshift(reverse(ifft(fft(arr1_sym) .* fft(reverse(arr2_sym)))), q0 .- 1)

show(stdout, "text/plain", expandKArr(kG, conv_test))
println("\n===============================")
show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_naive ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_fft_test ./ kG.Nk)

@test all(conv_ph_fft_test .≈ conv_ph_naive)
@test all(conv_ph_fft_test .≈ expandKArr(kG, conv_test) .* kG.Nk)

ttttttrue
4×4×4 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.00450261-0.431121im   0.0916384+0.0233439im    0.158434+0.0971723im   0.0916384+0.0233439im
   0.0916384+0.0233439im   0.106108-0.00438575im   0.112438-0.0305365im   0.0615202-0.180099im
    0.158434+0.0971723im   0.112438-0.0305365im   0.0371631-0.00624255im   0.112438-0.0305365im
   0.0916384+0.0233439im  0.0615202-0.180099im     0.112438-0.0305365im    0.106108-0.00438575im

[:, :, 2] =
 0.0916384+0.0233439im    0.106108-0.00438575im   0.112438-0.0305365im  0.0615202-0.180099im
  0.106108-0.00438575im    -0.2267-0.298624im    0.0996108+0.011892im    0.124766+0.0687761im
  0.112438-0.0305365im   0.0996108+0.011892im     0.105819+0.0508186im  0.0204225-0.0951959im
 0.0615202-0.180099im     0.124766+0.0687761im   0.0204225-0.0951959im   0.124766+0.0687761im

[:, :, 3] =
  0.158434+0.0971723im    0.112438-0.0305365im  0.0371631-0.00624255im   0.112438-0.0305365im
  0.112438-0.0305365im   0.0996108+0.011892im    0.105819+0.0508186im  

Test Passed

In [23]:
conv_pp_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=true);
conv_pp_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG, arr2_sym), crosscorrelation=false)
conv_pp_fft_test = circshift(ifft(fft(arr1_sym) .* fft(arr2_sym)), -0 .* q0 .+ 2)

#show(stdout, "text/plain", expandKArr(kG, conv_pp_test))
#println("\n===============================")
#show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
#println("\n===============================")
show(stdout, "text/plain", conv_pp_naive  ./ kG.Nk )
println("\n===============================")
show(stdout, "text/plain", conv_pp_fft_test ./ kG.Nk)

@test all(conv_pp_fft_test .≈ conv_pp_naive)
@test all(conv_pp_fft_test ./ kG.Nk .≈ expandKArr(kG, conv_pp_test))

tttttfalse
4×4×4 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.109669-0.162586im     0.105819+0.0508186im  0.0371631-0.00624255im   0.105819+0.0508186im
  0.105819+0.0508186im   0.0996108+0.011892im    0.112438-0.0305365im   0.0204225-0.0951959im
 0.0371631-0.00624255im   0.112438-0.0305365im   0.158434+0.0971723im    0.112438-0.0305365im
  0.105819+0.0508186im   0.0204225-0.0951959im   0.112438-0.0305365im   0.0996108+0.011892im

[:, :, 2] =
  0.105819+0.0508186im  0.0996108+0.011892im     0.112438-0.0305365im   0.0204225-0.0951959im
 0.0996108+0.011892im     -0.2267-0.298624im     0.106108-0.00438575im   0.124766+0.0687761im
  0.112438-0.0305365im   0.106108-0.00438575im  0.0916384+0.0233439im   0.0615202-0.180099im
 0.0204225-0.0951959im   0.124766+0.0687761im   0.0615202-0.180099im     0.124766+0.0687761im

[:, :, 3] =
 0.0371631-0.00624255im   0.112438-0.0305365im      0.158434+0.0971723im   0.112438-0.0305365im
  0.112438-0.0305365im    0.106108-0.00438575im    0.0916384+0.0233439im  0.0

LoadError: [91mThere was an error during testing[39m

In [9]:
@test all(arr1_c .== arr1_sym)
@test all(arr2_c .== arr2_sym)

Test Passed